In [1]:

#from omnia_ae.http_client import HttpClient, ContentType, RequestType
#from omnia_ae.models import (
#    SourceModel
#)
from typing import List, Literal, Optional
from azure.identity._internal.msal_credentials import MsalCredential

In [2]:
from typing import Literal, Optional, TypedDict, Union, Dict, Any
from azure.identity._internal.msal_credentials import MsalCredential
import requests
import logging

#from omnia_timeseries.helpers import retry
#from omnia_timeseries.models import TimeseriesRequestFailedException
from importlib import metadata
from opentelemetry.instrumentation.requests import RequestsInstrumentor
import platform

In [3]:
from azure.identity import ClientSecretCredential
import os
credential = ClientSecretCredential(
    tenant_id=os.environ['AZURE_TENANT_ID'],
    client_id=os.environ['AZURE_CLIENT_ID_AE_NON_PRODUCTION'],
    client_secret=os.environ['AZURE_CLIENT_SECRET_AE_NON_PRODUCTION']
)

KeyError: 'AZURE_CLIENT_ID_AE_NON_PRODUCTION'

In [3]:
base_url = "https://api-dev.gateway.equinor.com/iiot/ae/v1"

In [14]:
ContentType = Literal["application/json",
                      "application/protobuf", "application/x-google-protobuf"]

RequestType = Literal['get']
def _request(
    request_type: RequestType,
    url: str,
    headers: Dict[str, Any],
    payload: Optional[Union[TypedDict, dict, list]] = None,
    params: Optional[Dict[str, Any]] = None
) -> Union[Dict[str, Any], bytes]:

    response = requests.request(
        request_type, url, headers=headers, json=payload, params=params)
    #if not response.ok:
    #    raise TimeseriesRequestFailedException(response)
    if not "Accept" in headers or headers["Accept"] == "application/json":
        return response.json()
    else:
        return response.content

class HttpClient:
    def __init__(self, azure_credential: MsalCredential, resource_id: str):
        self._azure_credential = azure_credential
        self._resource_id = resource_id

    def request(
        self,
        request_type: RequestType,
        url: str,
        accept: ContentType = "application/json",
        payload: Optional[Union[TypedDict, dict, list]] = None,
        params: Optional[Dict[str, Any]] = None
    ) -> Any:

        access_token = self._azure_credential.get_token(
            f'{self._resource_id}/.default')  # handles caching and refreshing internally
        headers = {
            'Authorization': f'Bearer {access_token.token}',
            'Content-Type': 'application/json',
            'Accept': accept,
            ##'User-Agent': f'Omnia AE SDK/{version} {system_version_string}'
        }
        return _request(request_type=request_type, url=url, headers=headers, payload=payload, params=params)


In [11]:
resource_id="32f"

In [4]:
#check again on monday

In [16]:
print(base_url)

https://api-dev.gateway.equinor.com/iiot/ae/v1


In [24]:
access_token = credential.get_token(
    f'{resource_id}/.default')  # handles caching and refreshing internally
headers = {
    'Authorization': f'Bearer {access_token.token}',
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    ##'User-Agent': f'Omnia AE SDK/{version} {system_version_string}'
        }

ClientSecretCredential.get_token failed: Authentication failed: AADSTS7000215: Invalid client secret provided. Ensure the secret being sent in the request is the client secret value, not the client secret ID, for a secret added to app '657b2767-ee47-47ff-b745-af501ea053cb'. Trace ID: 49dd7a64-d026-40a6-ba9f-707aac60ba00 Correlation ID: d2efbacd-e893-4039-9655-e19646ea6bc7 Timestamp: 2023-11-10 15:03:16Z


ClientAuthenticationError: Authentication failed: AADSTS7000215: Invalid client secret provided. Ensure the secret being sent in the request is the client secret value, not the client secret ID, for a secret added to app '657b2767-ee47-47ff-b745-af501ea053cb'. Trace ID: 49dd7a64-d026-40a6-ba9f-707aac60ba00 Correlation ID: d2efbacd-e893-4039-9655-e19646ea6bc7 Timestamp: 2023-11-10 15:03:16Z

In [19]:
access_token

AccessToken(token='eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IjlHbW55RlBraGMzaE91UjIybXZTdmduTG83WSIsImtpZCI6IjlHbW55RlBraGMzaE91UjIybXZTdmduTG83WSJ9.eyJhdWQiOiIzMmYyYTkwOS04YTk4LTRlYjgtYjIyZC0xMjA4ZDkzNTBjYjAiLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8zYWE0YTIzNS1iNmUyLTQ4ZDUtOTE5NS03ZmNmMDViNDU5YjAvIiwiaWF0IjoxNjk5NjI3Mjg3LCJuYmYiOjE2OTk2MjcyODcsImV4cCI6MTY5OTYzMTE4NywiYWlvIjoiRTJWZ1lHaDgxSnBTVytiKzNGVjZyOFdxQTh4YkFBPT0iLCJhcHBpZCI6ImI1ZTc5YjBkLWZkZGItNGY2NS04ZDEwLTk5ODYzODU4Nzc5ZiIsImFwcGlkYWNyIjoiMSIsImdyb3VwcyI6WyJjNDNlNmQyOC0xZTM5LTQ1YWMtOTQ3YS03MDFjNWI5NzlhMDYiLCI4MDljYjA1YS1jNWVhLTRmZWItYjdmYi1kZmExMjIxMjViNzYiLCJkYjNhZTljYy1lYjAwLTRhMzctOWIxMi1hMWZhY2VmYTgyOTEiXSwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvM2FhNGEyMzUtYjZlMi00OGQ1LTkxOTUtN2ZjZjA1YjQ1OWIwLyIsIm9pZCI6ImIyY2ZhYjg0LTMxMDItNDQ3MS1iMmI1LTkyZTJmYjYxNWExNCIsInJoIjoiMC5BUUlBTmFLa091SzIxVWlSbFhfUEJiUlpzQW1wOGpLWWlyaE9zaTBTQ05rMURMQUNBQUEuIiwicm9sZXMiOlsiVGltZXNlcmllcy5BZG1pbiJdLCJzdWIiOiJiMmNmYWI4NC0zMTAyLTQ0NzEtYjJiNS05

In [18]:
_request(request_type='get', url=base_url+'/sources', headers=headers, payload=None, params=None)

{'statusCode': 401,
 'message': 'Unauthorized. Access token is missing or invalid.'}

In [ ]:
_http_client = HttpClient(
            azure_credential=azure_credential, resource_id=environment.resource_id)